# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad en Equipos: sistema LLM + RAG**

* **Nombres y matrículas:**

  *   Eduardo Aldair Ahumada Garcia Jurado - A01422929
  *   Angel Garcia Ortega - A01796653
  *   Ivan Troy Santaella Martínez - A01120515
  *   Diego Alberto Baños Lopez - A01275100

* **Número de Equipo:**
  * Equipo 63

# Introducción de la problemática a resolver

En un contexto global donde la protección de los datos personales se ha convertido en una prioridad tanto para los gobiernos como para las organizaciones, comprender y aplicar correctamente las normativas de protección de datos es un desafío constante. Cada país o región cuenta con leyes propias —como el GDPR en Europa, el CCPA en California, el LGPD en Brasil o la LFPDPPP en México— que imponen obligaciones específicas a quienes recaban, almacenan o procesan datos personales.

Sin embargo, estas normativas suelen estar redactadas en documentos extensos, complejos y en muchos casos dispersos, lo que dificulta su consulta y entendimiento, especialmente para profesionales no especializados en derecho. Esta situación representa un riesgo operativo y legal para las empresas, además de una barrera de conocimiento para estudiantes, profesionales de tecnología y ciudadanos que buscan comprender sus derechos.

Ante esta problemática, se propone el desarrollo de un chatbot inteligente basado en RAG (Retrieval-Augmented Generation) que permita consultar de forma sencilla y directa los textos legales oficiales de las principales leyes de protección de datos del mundo. Este asistente está diseñado para responder preguntas complejas de manera clara, precisa y contextualizada, basándose en los documentos legales reales y no únicamente en conocimientos generales del modelo.

El objetivo es democratizar el acceso al conocimiento jurídico, reducir el riesgo de interpretaciones erróneas y facilitar el cumplimiento normativo mediante una interfaz conversacional amigable, impulsada por modelos de lenguaje de última generación y técnicas de recuperación semántica.

# Tecnologías Utilizadas

- **OpenAI GPT-4o-mini**: modelo LLM económico y eficiente para generación de texto con alto nivel de comprensión y contexto.

- **OpenAI Embeddings (text-embedding-3-small)**: modelo de incrustaciones semánticas de bajo costo para vectorizar fragmentos de documentos.

- **LangChain**: framework para conectar modelos de lenguaje con flujos de trabajo como carga de documentos, embebido, recuperación y generación.

- **ChromaDB**: base de datos vectorial local para almacenamiento y recuperación eficiente de documentos embebidos.

- **Gradio**: interfaz visual interactiva para que el usuario pueda dialogar con el asistente desde un entorno accesible.

- **Jupyter Notebook**: plataforma utilizada para prototipar, integrar y visualizar todo el flujo de trabajo.



# Arquitectura del Sistema (RAG + LLM)

1. **Carga de documentos legales**: Se cargan documentos PDF oficiales (GDPR, CCPA, LGPD, LFPDPPP, etc.) desde una carpeta local.

2. **Fragmentación de texto**: Los documentos se dividen en fragmentos de texto pequeños (chunking) para optimizar su representación semántica.

3. **Embebido semántico**: Cada fragmento se convierte en un vector numérico utilizando text-embedding-3-small y se almacena en ChromaDB.

4. **Recuperación de contexto (Retriever)**: Cuando el usuario hace una pregunta, el sistema busca los fragmentos más relevantes utilizando búsqueda por similitud.

5. **Generación con contexto**: El modelo GPT-4o-mini recibe la pregunta y el contexto recuperado, generando una respuesta precisa, contextualizada y controlada por un prompt legal especializado.

6. **Respuesta y visualización**: El usuario recibe la respuesta junto con la fuente legal correspondiente, todo desde una interfaz Gradio en el cuaderno de Jupyter.

# Justificación y Relevancia

- **Alta utilidad práctica**: el chatbot puede ser utilizado por profesionales legales, estudiantes, analistas de datos, desarrolladores y responsables de cumplimiento normativo.

- **Reducción de riesgo**: evita interpretaciones erróneas de las leyes al usar solo información real contenida en los textos oficiales.

- **Escalabilidad**: la arquitectura permite agregar nuevos documentos legales de otros países o idiomas fácilmente.

- **Costo eficiente**: el uso de modelos livianos y vector stores locales permite mantener bajos los costos y operar en entornos personales.

# Evaluación Comparativa (LLM vs RAG + LLM)

Se realizaron cinco pruebas con preguntas reales sobre protección de datos. En todos los casos, el modelo RAG + LLM entregó respuestas más precisas, concisas y contextualizadas, mejorando sustancialmente la utilidad frente al uso directo del LLM sin contexto.

Para más informacion ve a la sección: ***Comparando RAG + LLM contra un LLM sin RAG***

# **Sistema RAG + LLM**

#### Setup API openAI

In [12]:
import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

#### Leer pdfs usando langchain

In [13]:
from langchain_community.document_loaders import PyPDFLoader
from pathlib import Path

# Load all PDF files in the `docs/` folder
def load_pdfs_from_folder(folder_path):
    all_docs = []
    for pdf_path in Path(folder_path).glob("*.pdf"):
        loader = PyPDFLoader(str(pdf_path))
        docs = loader.load()
        for doc in docs:
            doc.metadata["source"] = pdf_path.name  # Add source metadata
        all_docs.extend(docs)
    return all_docs

documents = load_pdfs_from_folder("docs/")
print(f"✅ Loaded {len(documents)} pages from PDF files.")
print(f"📄 First page preview:\n\n{documents[0].page_content[:500]}")


✅ Loaded 500 pages from PDF files.
📄 First page preview:

THE STATUTES OF THE REPUBLIC OF SINGAPORE
PERSONAL DATA PROTECTION
ACT 2012
2020 REVISED EDITION
This revised edition incorporates all amendments up to and
including 1 December 2021 and comes into operation on 31 December 2021.
Prepared and Published by
THE LAW REVISION COMMISSION
UNDER THE AUTHORITY OF
THE REVISED EDITION OF THE LAWS ACT 1983
Informal Consolidation– version in force from 1/10/2022


#### Separación de los documentos en chunks para el embedding y vector store

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Recommended for OpenAI embeddings: 1000 characters with 200 overlap
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)

# Apply to your loaded documents
chunks = text_splitter.split_documents(documents)

print(f"✅ Split into {len(chunks)} chunks.")
print(f"📄 First chunk preview:\n\n{chunks[0].page_content[:500]}")


✅ Split into 1887 chunks.
📄 First chunk preview:

THE STATUTES OF THE REPUBLIC OF SINGAPORE
PERSONAL DATA PROTECTION
ACT 2012
2020 REVISED EDITION
This revised edition incorporates all amendments up to and
including 1 December 2021 and comes into operation on 31 December 2021.
Prepared and Published by
THE LAW REVISION COMMISSION
UNDER THE AUTHORITY OF
THE REVISED EDITION OF THE LAWS ACT 1983
Informal Consolidation– version in force from 1/10/2022


#### Embeddding y creación del vector store

In [25]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# 1. Define embedding model
embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")

# 2. Set path to local DB
persist_directory = "chroma_db"

# 3. Try to load existing DB; if not, create from chunks
try:
    vectorstore = Chroma(
        embedding_function=embedding_function,
        persist_directory=persist_directory
    )
    print("✅ Loaded existing Chroma DB.")
except:
    print("🚧 Creating new vector store...")
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embedding_function,
        persist_directory=persist_directory
    )
    print("✅ New Chroma DB created.")

✅ Loaded existing Chroma DB.


#### Integración RAG + LLM 

In [16]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """You are a legal assistant specialized in international data protection laws such as the GDPR, CCPA, LGPD, APPI, and others.
Use only the information provided in the context to answer the user's question in a clear, concise, and professional manner.
If the context does not contain enough information, say that you don't have sufficient data to answer accurately.

User question:
{question}

Context:
{context}

Answer:"""
)


In [ ]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

# 1. Set up the LLM with gpt-4o-mini
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# 2. Create a retriever from the Chroma vector store
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

# 3. Build the Retrieval-Augmented Generation chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={'prompt': prompt_template}
)

# 4. Test it
query = "What are the legal bases for processing personal data under the GDPR?"
response = rag_chain.invoke(query)

# Show answer
print("🧠 Answer:\n", response["result"])

# Show sources
for doc in response["source_documents"]:
    print(f"\n📄 Source: {doc.metadata['source']}\n---\n{doc.page_content[:300]}")


🧠 Answer:
 The legal bases for processing personal data under the GDPR are as follows:

1. **Consent**: The data subject has given clear consent for their personal data to be processed for a specific purpose.
2. **Contractual necessity**: Processing is necessary for the performance of a contract to which the data subject is a party, or to take steps at the request of the data subject prior to entering into a contract.
3. **Legal obligation**: Processing is necessary for compliance with a legal or regulatory obligation to which the controller is subject.
4. **Vital interests**: Processing is necessary to protect the vital interests of the data subject or another natural person.
5. **Public task**: Processing is necessary for the performance of a task carried out in the public interest or in the exercise of official authority vested in the controller.
6. **Legitimate interests**: Processing is necessary for the purposes of legitimate interests pursued by the controller or a third party, 

# Comparando RAG + LLM contra un LLM sin RAG

In [18]:
query = "Does the GDPR allow processing without consent?"

# Without RAG (no context)
no_rag_response = llm.invoke(query)
print("🚫 No RAG Answer:\n", no_rag_response.content)

# With RAG
rag_response = rag_chain.invoke(query)
print("✅ RAG Answer:\n", rag_response["result"])


🚫 No RAG Answer:
 Yes, the General Data Protection Regulation (GDPR) allows for the processing of personal data without consent under certain circumstances. While consent is one of the legal bases for processing personal data, the GDPR outlines several other legal bases that organizations can rely on. These include:

1. **Contractual necessity**: Processing is necessary for the performance of a contract to which the data subject is a party or to take steps at the request of the data subject prior to entering into a contract.

2. **Legal obligation**: Processing is necessary for compliance with a legal obligation to which the data controller is subject.

3. **Vital interests**: Processing is necessary to protect the vital interests of the data subject or another natural person.

4. **Public task**: Processing is necessary for the performance of a task carried out in the public interest or in the exercise of official authority vested in the data controller.

5. **Legitimate interests**: 

In [19]:
query = "Is it mandatory to appoint a Data Protection Officer under Brazil's LGPD?"  # your question

# No RAG
no_rag_response = llm.invoke(query)
print("🚫 No RAG Answer:\n", no_rag_response.content)

# With RAG
rag_response = rag_chain.invoke(query)
print("✅ RAG Answer:\n", rag_response["result"])


🚫 No RAG Answer:
 Under Brazil's General Data Protection Law (Lei Geral de Proteção de Dados, or LGPD), the appointment of a Data Protection Officer (DPO) is not mandatory for all organizations. However, certain entities are required to appoint a DPO based on specific criteria.

According to the LGPD, a DPO is mandatory for:

1. Public authorities or bodies.
2. Organizations that process personal data on a large scale.
3. Organizations that process sensitive personal data or data related to children and adolescents.

Even if not required, organizations may choose to appoint a DPO to facilitate compliance with the LGPD and to serve as a point of contact for data subjects and the National Data Protection Authority (ANPD).

In summary, while not all organizations are required to appoint a DPO under the LGPD, certain types of organizations must do so, and others may benefit from having one.
✅ RAG Answer:
 Yes, under Brazil's LGPD (Lei Geral de Proteção de Dados), it is mandatory to appoint

In [21]:
query = "What are consumers' rights under the California CCPA?"  # your question

# No RAG
no_rag_response = llm.invoke(query)
print("🚫 No RAG Answer:\n", no_rag_response.content)

# With RAG
rag_response = rag_chain.invoke(query)
print("✅ RAG Answer:\n", rag_response["result"])


🚫 No RAG Answer:
 The California Consumer Privacy Act (CCPA), which took effect on January 1, 2020, grants California residents several rights regarding their personal information. Here are the key rights provided under the CCPA:

1. **Right to Know**: Consumers have the right to request that businesses disclose the categories and specific pieces of personal information they have collected about them, as well as the sources of that information, the purposes for which it is used, and the categories of third parties with whom it is shared.

2. **Right to Delete**: Consumers can request the deletion of their personal information that a business has collected, subject to certain exceptions (e.g., if the information is necessary for the business to complete a transaction or comply with a legal obligation).

3. **Right to Opt-Out**: Consumers have the right to opt-out of the sale of their personal information to third parties. Businesses must provide a clear and accessible way for consumers 

In [22]:
query = "Can personal data be transferred outside of Japan under APPI?"  # your question

# No RAG
no_rag_response = llm.invoke(query)
print("🚫 No RAG Answer:\n", no_rag_response.content)

# With RAG
rag_response = rag_chain.invoke(query)
print("✅ RAG Answer:\n", rag_response["result"])


🚫 No RAG Answer:
 Yes, under the Act on the Protection of Personal Information (APPI) in Japan, personal data can be transferred outside of Japan, but there are specific conditions that must be met to ensure the protection of that data.

1. **Consent**: The individual whose personal data is being transferred must provide consent for the transfer.

2. **Adequate Protection**: The receiving country or region must have a system in place that provides an adequate level of protection for personal data. The Japanese Personal Information Protection Commission (PPC) assesses whether a country offers adequate protection.

3. **Contractual Measures**: If the receiving country does not provide adequate protection, the data transfer can still occur if there are appropriate safeguards in place, such as contractual clauses that ensure the protection of personal data.

4. **Exceptions**: There are certain exceptions where personal data can be transferred without meeting the above conditions, such as 

In [24]:
query = "Is prior consent required for collecting biometric data in Mexico? based on the LFPDPPP"  # your question

# No RAG
no_rag_response = llm.invoke(query)
print("🚫 No RAG Answer:\n", no_rag_response.content)

# With RAG
rag_response = rag_chain.invoke(query)
print("✅ RAG Answer:\n", rag_response["result"])


🚫 No RAG Answer:
 Yes, under the Federal Law on Protection of Personal Data Held by Private Parties (LFPDPPP) in Mexico, prior consent is required for the collection of biometric data. Biometric data is considered sensitive personal data, and the law establishes that sensitive data can only be processed with the explicit consent of the individual. 

Organizations must inform individuals about the purpose of collecting their biometric data and obtain their consent before proceeding. Additionally, they must comply with other obligations set forth in the LFPDPPP, such as implementing adequate security measures to protect the data and allowing individuals to exercise their rights regarding their personal data.

For the most accurate and detailed information, it is advisable to consult the actual text of the LFPDPPP or seek legal counsel.
✅ RAG Answer:
 Yes, prior consent is required for collecting biometric data in Mexico under the Ley Federal de Protección de Datos Personales en Posesión 

1. El **modelo LLM solo** tiene buena base, pero puede ser demasiado generalista, especialmente en temas legales.

2. El **sistema RAG + LLM**:

    - Mejora la exactitud jurídica

    - Refuerza la confianza en la respuesta

    - Ofrece un estilo más claro y enfocado

    - Evita alucinaciones o respuestas poco fundamentadas



| Pregunta                                                               | ❌ LLM sin RAG                                                             | ✅ LLM con RAG                                                                                             | Ventaja de RAG                                                   |
| ---------------------------------------------------------------------- | ------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------- |
| **Does the GDPR allow processing without consent?**                    | Respuesta extensa, general y basada en conocimientos previos.             | Respuesta concisa, contextualizada y fundamentada en las excepciones del GDPR.                            | ✅ Mejora la precisión y se apoya en el documento real.           |
| **Is it mandatory to appoint a DPO under Brazil's LGPD?**              | Ambigua: afirma que depende del tipo de organización.                     | Directa: cita el artículo 41 de la LGPD que lo exige en todos los casos de tratamiento de datos.          | ✅ Cita la fuente legal y elimina ambigüedad.                     |
| **What are consumers' rights under the California CCPA?**              | Completa pero extensa, con mucha información redundante.                  | Clara y resumida, enumera los derechos principales con mejor foco.                                        | ✅ Mejora la claridad y evita sobreinformación.                   |
| **Can personal data be transferred outside of Japan under APPI?**      | Acierta, pero sin detalle normativo claro sobre consentimiento informado. | Precisa: indica el requisito de consentimiento y obligación de informar sobre el país receptor.           | ✅ Aporta contexto legal exacto y requisitos específicos.         |
| **Is prior consent required for collecting biometric data in Mexico?** | Acierta, pero de forma genérica y sin mencionar la ley específica.        | Precisa: cita explícitamente la LFPDPPP, habla del consentimiento libre e informado para datos sensibles. | ✅ Se alinea con la legislación mexicana y refuerza la respuesta. |


# Chatbot con Gradio para interecciones más amigables
***Escribe tu pregunta en la caja generada por Gradio donde dice "Your Question", una vez hecho esto da click en el botón "Ask" y espera unos segundos para que el modelo te responda.***

#### **Instrucciones para el Usuario Final**

Para utilizar este chatbot correctamente, el usuario debe seguir los siguientes pasos:

1. **Configurar el entorno**:

    - Crear un archivo .env en el directorio raíz del proyecto.

    - Incluir su clave API de OpenAI en el archivo con el siguiente formato:

        -  OPENAI_API_KEY=sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

2. **Documentación utilizada**:

    - El sistema se basa en los siguientes documentos legales oficiales:

        - GDPR (Unión Europea)

        - CCPA (California, EE.UU.)

        - LGPD (Brasil)

        - LFPDPPP (México)

        - APPI (Japón)

    - Todos los archivos PDF deben ser colocados en la carpeta docs/ antes de ejecutar el sistema.

3. **Ejecución del chatbot**:

    - Abrir el Jupyter Notebook del proyecto.

    - Ejecutar las celdas paso a paso hasta que se despliegue la interfaz Gradio.

    - Ingresar consultas en inglés relacionadas con las leyes mencionadas.

4. **Limitaciones**:

    - El chatbot responde únicamente con base en los documentos cargados.

    - Si no encuentra suficiente información relevante, informará que no puede dar una respuesta precisa.

5. **Recomendaciones**:

    - Formular preguntas claras y específicas.

    - Mencionar el país o ley cuando sea posible para mejorar la recuperación de contexto.

***Este asistente está diseñado para facilitar la comprensión legal, pero no sustituye asesoría jurídica profesional.***

In [27]:
import gradio as gr

# RAG function to connect UI to the model
def ask_data_law(query):
    if not query.strip():
        return "Please enter a valid question."
    
    response = rag_chain.invoke(query)
    answer = response["result"]

    # show sources (for transparency)
    sources = "\n\n".join(
        f"📄 {doc.metadata['source']}" for doc in response["source_documents"]
    )
    
    return f"🧠 **Answer:**\n{answer}\n\n---\n**Sources:**\n{sources}"

# Gradio Blocks interface
with gr.Blocks() as demo:
    gr.Markdown("# 💼 Ask the Data Law Assistant")
    gr.Markdown("Ask about GDPR, CCPA, LGPD, APPI, and more.")

    with gr.Row():
        textbox = gr.Textbox(label="Your Question")
        submit_btn = gr.Button("Ask")

    output = gr.Markdown()

    submit_btn.click(fn=ask_data_law, inputs=textbox, outputs=output)

# Launch the app inline
demo.launch(inline=True, share=False)


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


# **Conclusiones:**

#### **Conclusiones de la actividad chatbot LLM + RAG:**
El desarrollo de este chatbot legal demuestra de forma concreta el potencial de las arquitecturas RAG (Retrieval-Augmented Generation) combinadas con LLMs para resolver desafíos complejos relacionados con la consulta de documentos normativos. A diferencia de un modelo de lenguaje tradicional, el sistema aquí presentado se apoya directamente en documentación legal oficial para responder, lo cual mejora la precisión, reduce los riesgos de alucinación y fortalece la trazabilidad de cada respuesta.

Este enfoque es especialmente útil en dominios donde la exactitud del contenido es crítica, como en el derecho, la medicina, las políticas públicas o la educación. El asistente legal no sólo brinda respuestas claras y fundamentadas, sino que también puede ser escalado a otras jurisdicciones, adaptado a nuevos marcos regulatorios, y extendido a otras lenguas si se integran nuevos documentos fuente.

Además, se valida que con herramientas accesibles y de bajo costo como GPT-4o-mini, embeddings optimizados como text-embedding-3-small, y un entorno local basado en LangChain y ChromaDB, es posible construir asistentes conversacionales avanzados sin depender de infraestructura compleja o costosa.

Este proyecto representa un caso de uso realista, educativo y replicable que puede ser reutilizado en instituciones académicas, equipos legales internos, ONGs o pequeñas empresas que necesiten asistencia legal automatizada sin renunciar a la fiabilidad.

# **Fin de la actividad chatbot: LLM + RAG**